<a href="https://colab.research.google.com/github/EbsHirani/MovieRecommendation/blob/master/Notflix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
os.chdir("drive/My Drive/Movierecommend")

In [0]:
!unzip \*.zip


Archive:  Collaborative Filtering Dataset.zip
   creating: dataset/
  inflating: dataset/links.csv       
   creating: __MACOSX/
   creating: __MACOSX/dataset/
  inflating: __MACOSX/dataset/._links.csv  
  inflating: dataset/tags.csv        
  inflating: __MACOSX/dataset/._tags.csv  
  inflating: dataset/ratings.csv     
  inflating: __MACOSX/dataset/._ratings.csv  
  inflating: dataset/README.txt      
  inflating: __MACOSX/dataset/._README.txt  
  inflating: dataset/movies.csv      
  inflating: __MACOSX/dataset/._movies.csv  


In [0]:
os.chdir("dataset")

In [4]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
ratings_pd = pd.read_csv("ratings.csv")
ratings_pd.head()


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [33]:
movies = pd.read_csv("movies.csv")
movies["movieId"] = movies["movieId"].astype("str")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
link = pd.read_csv("links.csv")
link.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [7]:
tag = pd.read_csv("tags.csv")
tag.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [0]:
ratings_pd["userId"] = ratings_pd["userId"].astype("str")
ratings_pd["movieId"] = ratings_pd["movieId"].astype("str")

In [9]:
ratings = ratings_pd.pivot(index = "userId", columns = "movieId", values = "rating")
ratings.head()


movieId,1,10,100,100044,100068,100083,100106,100159,100163,100194,100226,100277,1003,100302,100304,100306,100326,100383,100390,100397,1004,100487,100498,1005,100507,100527,100553,100556,100579,1006,100611,1007,100714,100737,1008,100810,100843,100882,1009,100906,...,98836,98908,98961,99,990,99005,99007,99030,99087,991,99106,99112,99114,99117,99122,99130,99145,99149,99191,993,994,99415,99437,99532,99574,996,99636,99638,99721,99728,99750,99764,998,99813,99846,99853,999,99910,99917,99992
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
def standardise(row):
  return (row - row.mean())/(row.max() - row.min())

In [11]:
ratings = ratings.fillna(0)
ratings_std = ratings.apply(standardise, axis = 1)
ratings_std.head()

movieId,1,10,100,100044,100068,100083,100106,100159,100163,100194,100226,100277,1003,100302,100304,100306,100326,100383,100390,100397,1004,100487,100498,1005,100507,100527,100553,100556,100579,1006,100611,1007,100714,100737,1008,100810,100843,100882,1009,100906,...,98836,98908,98961,99,990,99005,99007,99030,99087,991,99106,99112,99114,99117,99122,99130,99145,99149,99191,993,994,99415,99437,99532,99574,996,99636,99638,99721,99728,99750,99764,998,99813,99846,99853,999,99910,99917,99992
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.779165,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,0.579165,-0.020835,...,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835,-0.020835
10,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,...,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441,-0.009441
100,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,...,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012,-0.012012
101,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,...,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463,-0.004463
102,-0.003867,-0.003867,-0.003867,-0.003867,-0.003867,-0.003867,-0.003867,-0.003867,-0.003867,-0.003867,-0.003867,-0.003867,-0.003867,-0.003867,-0.003867,-0.003867,-0.003867,-0.003867,-0.003867,-0

Making recommendations of similar movies using cosine similarity:


In [12]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
movie_similarity = cosine_similarity(ratings_std.T)
print(movie_similarity)


[[ 1.          0.36186182 -0.02470436 ... -0.19707874 -0.15300245
  -0.324391  ]
 [ 0.36186182  1.          0.0268841  ... -0.15647941 -0.16216091
  -0.22110123]
 [-0.02470436  0.0268841   1.         ... -0.01933622  0.06909337
   0.00415767]
 ...
 [-0.19707874 -0.15647941 -0.01933622 ...  1.          0.02364279
   0.16231011]
 [-0.15300245 -0.16216091  0.06909337 ...  0.02364279  1.
   0.08924925]
 [-0.324391   -0.22110123  0.00415767 ...  0.16231011  0.08924925
   1.        ]]


In [13]:
movie_similarity_df = pd.DataFrame(movie_similarity, index = ratings.columns, columns = ratings.columns)
movie_similarity_df.head()

movieId,1,10,100,100044,100068,100083,100106,100159,100163,100194,100226,100277,1003,100302,100304,100306,100326,100383,100390,100397,1004,100487,100498,1005,100507,100527,100553,100556,100579,1006,100611,1007,100714,100737,1008,100810,100843,100882,1009,100906,...,98836,98908,98961,99,990,99005,99007,99030,99087,991,99106,99112,99114,99117,99122,99130,99145,99149,99191,993,994,99415,99437,99532,99574,996,99636,99638,99721,99728,99750,99764,998,99813,99846,99853,999,99910,99917,99992
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.361862,-0.024704,-0.314082,-0.279652,-0.175216,-0.343308,-0.218164,-0.057715,-0.280527,-0.458655,-0.314082,-0.209852,-0.373012,-0.324391,-0.261008,-0.367965,-0.066209,-0.187358,-0.281186,-0.069984,-0.414083,-0.119653,-0.062680,-0.138405,-0.294978,-0.223768,-0.231690,-0.247403,-0.108463,-0.226875,0.030119,-0.206638,-0.261495,-0.314099,-0.251178,-0.292178,-0.198882,0.002402,-0.192932,...,-0.414083,-0.458655,0.014674,-0.100760,-0.106093,-0.458655,0.008869,-0.247403,-0.315368,0.016647,-0.221473,0.059953,0.192867,-0.040100,-0.288000,-0.288000,-0.183998,-0.048595,-0.230745,-0.357853,0.099767,-0.414083,-0.192086,-0.318374,-0.324391,-0.050702,-0.247173,-0.366166,-0.453672,-0.128544,-0.288878,-0.133258,-0.144334,-0.041811,-0.324391,-0.258072,-0.003184,-0.197079,-0.153002,-0.324391
10,0.361862,1.000000,0.026884,-0.239515,-0.215110,-0.171715,-0.261332,-0.230456,-0.053298,-0.213110,-0.330088,-0.239515,-0.181872,-0.283122,-0.221101,-0.244419,-0.255773,-0.119285,-0.149336,-0.273347,-0.064394,-0.293078,-0.032694,-0.030848,-0.146169,-0.222375,-0.173375,-0.126568,-0.250549,-0.038374,-0.171591,0.042972,-0.157794,-0.244929,-0.226949,-0.218003,-0.240776,-0.072946,0.023927,-0.212872,...,-0.293078,-0.330088,0.052511,-0.149959,-0.092175,-0.330088,-0.034267,-0.250549,-0.241655,0.087709,-0.171411,0.086464,0.072873,-0.065312,-0.250298,-0.250298,-0.165426,-0.116380,-0.223881,-0.321964,-0.042416,-0.293078,-0.187580,-0.177107,-0.221101,0.051003,-0.185898,-0.241261,-0.378588,-0.152234,-0.265730,-0.164835,-0.127075,-0.032102,-0.221101,-0.229329,0.138905,-0.156479,-0.162161,-0.221101
100,-0.024704,0.026884,1.000000,0.009151,0.019153,-0.015832,0.013916,-0.008440,-0.048012,0.014963,0.033096,0.009151,0.046288,0.021954,0.004158,0.031817,0.012461,-0.027073,-0.007749,0.001579,0.283549,0.022198,-0.021320,-0.037247,-0.004849,0.030001,-0.016219,0.017096,-0.003894,0.386222,0.018438,-0.028218,0.004238,0.031268,0.019424,0.009826,0.035760,0.011398,0.043688,-0.012234,...,0.022198,0.033096,-0.073536,-0.000897,0.141130,0.033096,-0.027553,-0.003894,0.024381,0.117820,0.000693,-0.090114,-0.088468,-0.053737,0.025520,0.025520,-0.004667,-0.010761,0.028138,0.154162,0.000829,0.022198,-0.009237,0.140772,0.004158,0.325971,0.028784,0.156907,0.035386,-0.029612,0.009365,0.011925,0.010135,0.006722,0.004158,0.021451,0.130583,-0.019336,0.069093,0.004158
100044,-0.314082,-0.239515,0.009151,1.000000,0.258180,0.100029,0.997464,0.166837,0.018013,0.212212,0.397197,1.000000,0.110356,0.287208,0.258518,0.280679,0.302258,0.028585,0.121125,0.227375,0.120449,0.349656,0.110743,0.049274,0.107100,0.258044,0.714799,0.201571,0.194822,0.032926,0.187126,-0.064416,0.139721,0.279698,0.283086,0.203137,0.287736,0.152382,-0.018753,0.142813,...,0.349656,0.397197,-0.111883,0.120252,0.083527,0.397197,-0.080566,0.194822,0.291175,-0.173732,0.182605,-0.131358,-0.092859,-0.024062,0.269460,0.269460,0.123298,0.067717,0.254535,0.233914,-0.147407,0.349656,0.129773,0.212933,0.258518,-0.046756,0.225143,0.242297,0.399392,0.083392,0.240963,0.165034,0.168897,-0.024059,0.258518,0.241974,-0.095300,0.141831,0.082199,0.258518
100068,-0.279652,-0.215110,0.019153,0.258180,1.000000,0.123027,0.292594,0.200615,0.033667,0.248626,0.464055,0.258180,0.140941,0.336057,0.306465,0.325775,0.356380,0.040905,0.146061,0.270075,0.142746,0.410280,0.137172,0.068110,0.128639,0.299313,0.123566,0.235424,0.232746,0.047979,0.217899,-0.057203,0.165128,0.324749,0.331798,0.23914

Using pearson's correlation:


In [44]:
movie_similarity_df_pearson = ratings.corr(method = 'pearson' )
movie_similarity_df_pearson.head()

movieId,1,10,100,100044,100068,100083,100106,100159,100163,100194,100226,100277,1003,100302,100304,100306,100326,100383,100390,100397,1004,100487,100498,1005,100507,100527,100553,100556,100579,1006,100611,1007,100714,100737,1008,100810,100843,100882,1009,100906,...,98836,98908,98961,99,990,99005,99007,99030,99087,991,99106,99112,99114,99117,99122,99130,99145,99149,99191,993,994,99415,99437,99532,99574,996,99636,99638,99721,99728,99750,99764,998,99813,99846,99853,999,99910,99917,99992
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.183382,0.033699,-0.028906,0.054760,0.009749,-0.028906,0.075676,0.126465,-0.028906,0.075676,-0.028906,0.090236,-0.028906,0.075676,0.065218,0.075676,0.028172,-0.008055,0.075676,0.073309,0.075676,0.104770,0.116454,0.007007,-0.028906,-0.040912,0.002469,0.075676,0.030928,-0.040595,0.165821,-0.046210,0.065218,0.065218,0.007995,0.012927,-0.040856,0.113136,0.075676,...,0.075676,0.075676,0.032643,0.077506,0.059054,0.075676,0.047372,0.075676,0.054760,0.101023,0.040856,0.131970,0.063904,0.074229,0.033843,0.033843,-0.010412,0.042387,0.065218,0.054760,0.161932,0.075676,0.023663,0.050116,0.075676,0.038390,-0.028906,0.033843,0.075676,0.096050,0.054760,0.067748,0.065845,0.035172,0.075676,0.033843,0.075123,0.090900,0.088073,0.075676
10,0.183382,1.000000,0.076649,-0.020524,0.033732,-0.033599,-0.020524,-0.020524,0.084107,-0.020524,0.087987,-0.020524,0.044227,-0.020524,0.087987,-0.020524,0.087987,-0.049879,-0.011791,-0.020524,0.037784,0.087987,0.143707,0.103950,-0.040499,-0.020524,-0.029049,0.060859,-0.020524,0.073873,-0.028824,0.144830,-0.032810,-0.020524,0.060859,-0.027553,-0.020524,0.061803,0.108081,-0.020524,...,0.087987,0.087987,0.078375,-0.029049,0.030810,0.087987,-0.003726,-0.020524,0.033732,0.159084,0.025798,0.144397,-0.026421,0.021022,-0.020524,-0.020524,-0.035100,-0.053937,-0.020524,-0.020524,-0.000587,0.087987,-0.028824,0.118124,0.087987,0.129420,-0.020524,0.074423,-0.020524,0.011261,-0.020524,-0.028605,0.024543,0.029451,0.087987,-0.020524,0.208809,0.059381,0.016366,0.087987
100,0.033699,0.076649,1.000000,-0.006033,-0.006033,-0.009876,-0.006033,-0.006033,-0.012310,-0.006033,-0.006033,-0.006033,0.073925,-0.006033,-0.006033,-0.006033,-0.006033,-0.014661,-0.011167,-0.006033,0.276581,-0.006033,-0.013874,-0.015858,-0.011904,-0.006033,-0.008538,-0.006033,-0.006033,0.400070,-0.008472,0.026269,-0.009644,-0.006033,-0.006033,-0.008099,-0.006033,-0.008527,0.073194,-0.006033,...,-0.006033,-0.006033,-0.022521,-0.008538,0.147775,-0.006033,0.013286,-0.006033,-0.006033,0.191084,-0.009946,-0.020577,-0.031971,-0.016493,-0.006033,-0.006033,-0.010317,-0.015854,-0.006033,0.182678,0.068082,-0.006033,-0.008472,0.160800,-0.006033,0.353963,-0.006033,0.182678,-0.006033,-0.010287,-0.006033,-0.008408,-0.009463,0.034167,-0.006033,-0.006033,0.177242,-0.008513,0.092664,-0.006033
100044,-0.028906,-0.020524,-0.006033,1.000000,-0.001642,-0.002688,1.000000,-0.001642,-0.003351,-0.001642,-0.001642,1.000000,-0.002741,-0.001642,-0.001642,-0.001642,-0.001642,-0.003991,-0.003040,-0.001642,-0.003965,-0.001642,-0.003776,-0.004316,-0.003240,-0.001642,0.706526,-0.001642,-0.001642,-0.003596,-0.002306,-0.005084,-0.002625,-0.001642,-0.001642,-0.002204,-0.001642,-0.002321,-0.004837,-0.001642,...,-0.001642,-0.001642,-0.006130,-0.002324,-0.003368,-0.001642,-0.006067,-0.001642,-0.001642,-0.005355,-0.002707,-0.005601,0.141370,-0.004489,-0.001642,-0.001642,-0.002808,-0.004315,-0.001642,-0.001642,-0.005418,-0.001642,-0.002306,-0.002204,-0.001642,-0.005733,-0.001642,-0.001642,-0.001642,-0.002800,-0.001642,-0.002289,-0.002576,-0.004587,-0.001642,-0.001642,-0.005583,-0.002317,-0.002841,-0.001642
100068,0.054760,0.033732,-0.006033,-0.001642,1.000000,-0.002688,-0.001642,-0.001642,-0.003351,-0.001642,-0.001642,-0.001642,-0.002741,-0.001642,-0.001642,-0.001642,-0.001642,-0.003991,-0.003040,-0.001642,-0.003965,-0.001642,-0.003776,-0.004316,-0.003240,-0.001642,-0.002324,-0.001642,-0.001642,-0.003596,-0.002306,-0.005084

In [17]:
def get_similar(movie_id, rating):
  scores = movie_similarity_df[movie_id]*(rating - 3) #Ratings below 3 are considered bad so negative cosine similarities will be at top
  return scores
get_similar("100",5)

movieId
1        -0.049409
10        0.053768
100       2.000000
100044    0.018302
100068    0.038306
            ...   
99853     0.042903
999       0.261165
99910    -0.038672
99917     0.138187
99992     0.008315
Name: 100, Length: 9724, dtype: float64

In [22]:
def suggest_movies(movie_list): #movie_list format[(movie_id1,rating1),(movie_id2,rating2)....]
  similar = pd.DataFrame()
  for movie, rating in movie_list:
    similar = similar.append(get_similar(movie, rating))
  return similar.sum().sort_values(ascending = False)
sim = suggest_movies([('100', 5), ("1",4)])
sim.head()

100    1.975296
1      0.950591
26     0.919884
74     0.813829
76     0.798996
dtype: float64

In [36]:
def get_movie_by_id(movie_id):
   return (movies["title"][movies["movieId"] == movie_id].values[0])
print(get_movie_by_id("1"))

Toy Story (1995)


In [0]:
movie_idx = {movie_name : movie_id for movie_name, movie_id in movies[["title", "movieId"]].values}
print(movie_idx)

Suggesting through users with similar taste:

In [45]:
user_corr = ratings.T.corr(method = "pearson")
user_corr.head()

userId,1,10,100,101,102,103,104,105,106,107,108,109,11,110,111,112,113,114,115,116,117,118,119,12,120,121,122,123,124,125,126,127,128,129,13,130,131,132,133,134,...,63,64,65,66,67,68,69,7,70,71,72,73,74,75,76,77,78,79,8,80,81,82,83,84,85,86,87,88,89,9,90,91,92,93,94,95,96,97,98,99
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,-0.000298,0.131782,0.131810,0.147350,0.187823,0.098766,0.051680,0.010849,0.058849,0.045639,0.136919,0.122323,0.134142,0.035804,0.158071,0.055668,0.070954,0.213273,0.174719,0.148595,0.126881,0.035610,0.007960,0.105861,0.107976,0.172976,0.075206,0.191187,0.062900,0.123526,0.035710,0.151210,0.172167,0.085803,0.073665,0.140594,0.138450,0.150832,0.134153,...,0.196571,0.291700,0.056447,0.211442,0.111466,0.229394,0.145954,0.143793,0.067045,0.091449,0.202524,0.028283,0.043684,0.132664,0.170327,0.091245,0.169118,0.207402,0.128547,0.046441,0.103325,0.197902,0.097375,0.143293,0.007745,0.136680,0.025472,0.061222,-0.007874,0.055268,0.041326,0.311346,0.021590,0.221377,0.100556,0.161682,0.273976,0.078530,0.048744,0.056308
10,-0.000298,1.000000,0.077805,0.012060,0.028124,0.097400,0.163043,0.124151,0.127689,-0.003455,0.033838,0.017023,0.018553,0.005164,0.145091,0.109545,0.024119,0.063216,0.026513,0.050338,0.011572,0.000047,0.190805,0.085446,-0.005221,0.018456,0.055178,0.048143,0.055320,0.065576,0.023643,-0.001899,0.012732,0.113907,0.032235,0.009306,0.074163,0.142020,0.032343,0.011463,...,0.130024,0.050617,0.067942,0.067150,0.076328,0.188875,0.011800,0.120606,0.037488,-0.006535,0.030208,0.163790,0.003889,0.073090,0.044920,0.180409,0.007105,-0.001927,0.016471,0.076503,0.023347,0.061503,0.150526,0.014602,-0.006333,0.146387,-0.005196,0.094777,0.029459,0.033996,-0.008333,0.029846,0.011317,0.018060,0.030189,0.028712,0.009334,0.051441,0.120950,0.012731
100,0.131782,0.077805,1.000000,0.062296,0.134411,0.115666,0.161727,0.034055,-0.007043,0.091950,0.110873,0.143308,0.100593,0.021811,0.045113,0.105208,0.137608,0.043954,0.125827,0.168182,0.170429,0.112935,0.047493,0.126244,0.009361,0.194534,0.090316,0.012608,0.107439,0.022757,0.073529,0.003763,0.017435,0.072353,0.050035,0.045769,0.040356,0.195376,0.112421,0.061020,...,0.104911,0.175063,0.038282,0.158830,0.089729,0.204913,0.069491,0.109719,0.071458,0.066931,0.074157,0.027624,0.000035,0.022034,0.049465,0.022439,0.081237,0.062517,0.130250,0.022617,0.061167,0.108612,0.071974,0.235770,-0.006824,0.081056,0.006194,0.038647,0.017239,0.051150,0.006311,0.166455,-0.006017,0.111668,0.117334,0.079222,0.061337,0.071503,0.042154,0.071191
101,0.131810,0.012060,0.062296,1.000000,0.021262,0.093078,0.033505,0.046679,-0.004380,-0.004434,0.029178,0.019598,0.011820,0.081336,0.029994,0.040875,0.005768,0.020429,0.125508,0.027783,-0.009840,-0.003545,0.024902,0.012216,0.038304,-0.005722,0.066294,0.036021,0.075948,0.048782,-0.004430,-0.003336,-0.004392,0.101199,0.085002,-0.003977,0.013928,0.092556,0.018409,-0.004487,...,0.095669,0.144063,0.028458,0.108865,0.050421,0.096563,0.082407,0.053480,0.030321,0.014308,0.070780,0.034888,0.045677,-0.005996,0.077153,0.057267,0.028590,0.062768,-0.005127,0.017105,-0.003641,0.072649,0.055381,0.006472,0.033485,0.095717,-0.003482,0.027627,0.024775,0.018518,-0.005583,0.119814,-0.003742,0.005481,-0.005506,0.061666,0.101626,0.099638,-0.007163,-0.005387
102,0.147350,0.028124,0.134411,0.021262,1.000000,0.081628,0.076676,0.032850,0.025808,0.351097,0.006651,0.490065,0.273221,0.058701,0.031831,0.206957,0.048529,0.019607,0.178502,0.258256,0.530190,0.085679,0.067153,0.014357,0.024745,0.490064,0.098101,0.103614,0.138079,0.060110,0.598033,-0.003243,0.026403,0.043665,0.042277,0.377917,0.126121,0.133746,0.528356,0.520892,...,0.141832,0.175511,0.021930,0.116511,0.143000,0.134502,0.151444,0.105420,0.050801,0.084867,0.276954,-0.006387,0.007085,0.054337,0.125791,-0.003853,0.234822,0.079303,0.509100,0.027066,0.512680,0.167882,0.075500,0.234285,-0.004126,0.119549,-0.003385,0.047370,-0.013475,0.028085,-0.005428,0.249751,-0.003638,0.214253,0.5842

In [59]:
def find_similar_users(user_id):
  return user_corr[user_id].sort_values(ascending = False).index[1:]
print(find_similar_users("1"))



Index(['266', '313', '368', '57', '39', '452', '91', '469', '45', '135',
       ...
       '175', '397', '578', '358', '89', '496', '556', '506', '184', '306'],
      dtype='object', name='userId', length=609)


In [61]:
THRESH = 3
def suggest_user_based_movies(user_id):
  user_list = find_similar_users(user_id)[:THRESH]
  movieId_list = []
  for userId in user_list:
    #print(userId)
    movieId_list = movieId_list + list(ratings.loc[userId].sort_values(ascending = False).index[:THRESH].astype("str"))
    #print(movieId_list)
  return movieId_list
movieId_list = suggest_user_based_movies("1")
for i in movieId_list:
  print(get_movie_by_id(i))    


Brazil (1985)
Election (1999)
Heathers (1989)
Silence of the Lambs, The (1991)
Gladiator (2000)
Road Warrior, The (Mad Max 2) (1981)
Apocalypse Now (1979)
Aliens (1986)
Saving Private Ryan (1998)
